In [1]:
import requests as r
from datetime import datetime
import os

###### Functoin to call earthdata API to get the data

In [2]:
# "product" : products required from GEDI data ('GEDI01_B.002', 'GEDI02_A.002' and 'GEDI02_B.002')
# "bbox" :  bbox is a string of bounding box coordinate values in decimal degrees. 
            #(Lower Left Longitude, Lower Left Latitude, Upper Right Longitude, Upper Right Latitude)
    

def gedi_finder(product, bbox):

    # Define the base CMR granule search url, including LPDAAC provider name and max page size (2000 is the max allowed)
    cmr = "https://cmr.earthdata.nasa.gov/search/granules.json?pretty=true&provider=LPDAAC_ECS&page_size=2000&concept_id="

    # Set up dictionary where key is GEDI shortname + version
    # A dictionary is set up to relate each product "shortname.version" to its associated "concept_id",
    # which is a value used by NASA's CMR to retrieve data for a specific product. 
    concept_ids = {'GEDI01_B.002': 'C1908344278-LPDAAC_ECS', 
                   'GEDI02_A.002': 'C1908348134-LPDAAC_ECS', 
                   'GEDI02_B.002': 'C1908350066-LPDAAC_ECS'}

    # CMR uses pagination for queries with more features returned than the page size
    page = 1
    bbox = bbox.replace(' ', '')  # remove any white spaces
    try:
        # Send GET request to CMR granule search endpoint w/ product concept ID, bbox & page number, format return as json
        cmr_response = r.get(f"{cmr}{concept_ids[product]}&bounding_box={bbox}&pageNum={page}").json()['feed']['entry']

        # If 2000 features are returned, move to the next page and submit another request, and append to the response
        while len(cmr_response) % 2000 == 0:
            page += 1
            cmr_response += r.get(f"{cmr}{concept_ids[product]}&bounding_box={bbox}&pageNum={page}").json()['feed']['entry']

        # CMR returns more info than just the Data Pool links, below use list comprehension to return a list of DP links
        return [c['links'][0]['href'] for c in cmr_response]
    except:
        # If the request did not complete successfully, print out the response from CMR
        print(r.get(f"{cmr}{concept_ids[product]}&bounding_box={bbox.replace(' ', '')}&pageNum={page}").json())

###### Call the function with coordinates (Longitude-Lattitude) for AOI

In [3]:
# User-provided inputs (UPDATE FOR YOUR DESIRED PRODUCT AND BOUNDING BOX REGION OF INTEREST)
product = 'GEDI01_B.002'           # Options include 'GEDI01_B.002', 'GEDI02_A.002', 'GEDI02_B.002'
bbox = '-77.6886,43.0791,-77.6634,43.0897'  # bounding box coordinates in LL Longitude, LL Latitude, UR Longitude, UR Latitude format

In [4]:
granules = gedi_finder(product, bbox)
print(f"Total {len(granules)} {product} Version 2 granules found.")

Total 21 GEDI01_B.002 Version 2 granules found.


###### Export Data

In [10]:
# Set up output text file name using the current datetime
outName =f"{product.replace('.', '_')}_GranuleList_{datetime.now().strftime('%Y%m%d%H%M%S')}.txt"

# Open file and write each granule link on a new line
with open(outName, "w") as gf:
    for g in granules:
        gf.write(f"{g}\n")
print(f"File containing links to intersecting {product} Version 2 data has been saved to:\n {os.getcwd()}\{outName}")

File containing links to intersecting GEDI01_B.002 Version 2 data has been saved to:
 D:\Study\Spring_23\CSCI_788\Leveraging-Satellite-LiDAR-Data-for-3D-Reconstruction\GEDI_Finder\GEDI01_B_002_GranuleList_20230305033303.txt
